# Navigation

---

This notebook is a solution to the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
from p1_navigation.agents import DQN_Agent, DoubleDQN_Agent, Dueling_DDQN_Agent
import matplotlib.pyplot as plt
import torch
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="C:\Dev\Python\RL\deep-reinforcement-learning\p1_navigation\Banana_Windows_x86_64\Banana.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Take Actions and learn in the Environment

In [ ]:
def train_simulation(Agent, num_episodes=1500, num_sims=5, eps_start=1.0, eps_end=0.005, eps_decay=0.995):
    """Run simulations in the environment and learn from the experiences. This will train the Agent.

    :param Agent: (class) This is the class of Agent from agents.py that you want to use
    :param num_episodes: (int) maximum number of training episodes
    :param num_sims: (int) How many simulations to run for this Agent to get average scores
    :param eps_start: (float) starting value of epsilon, for epsilon-greedy action selection
    :param eps_end: (float) minimum value of epsilon
    :param eps_decay: (float) multiplicative factor (per episode) for decreasing epsilon
    :return: list of scores achieved over the simulation
    """
    scores = [0]*num_episodes
    for sim in range(1,num_sims+1):
        # Reset the Agent and epsilon
        agent = Agent(state_size, action_size, seed=88)
        eps = eps_start
        for episode_num in range(1,num_episodes+1):
            env_info = env.reset(train_mode=True)[brain_name]
            state = env_info.vector_observations[0]
            score = 0
            while True:
                action = int(agent.act(state,eps))
                env_info = env.step(action)[brain_name]
                next_state = env_info.vector_observations[0]
                reward = env_info.rewards[0]
                done = env_info.local_done[0]
                agent.step(state, action, reward, next_state, done)
                score += reward
                state = next_state
                if done:
                    break
            # Episode finished
            scores[episode_num-1] += score
            eps = max(eps*eps_decay, eps_end)
            print(f"Sim: {sim}, Episode: {episode_num}, Score: {score}")
    #Average scores for the number of simulations
    scores = [score/num_sims for score in scores]

    # All episodes finished
    torch.save(agent.qnetwork_local.state_dict(), f"C:\Dev\Python\RL\\Navigation_Project\p1_navigation\checkpoints\\{agent}.pth")
    #Save scores to file so we do not need to rerun the entire simulation
    f = open(f"C:\Dev\Python\RL\\Navigation_Project\p1_navigation\saved_scores\\{agent}.txt", 'w')
    s1 = '\n'.join([str(s) for s in scores])
    f.write(s1)
    f.close()
    
    return scores

# Setup what sort of simulation you want
train_dqn = False
train_double_dqn = False
train_dueling = False
if train_dqn:
    train_simulation(DQN_Agent)
if train_double_dqn:
    train_simulation(DoubleDQN_Agent)
if train_dueling:
    train_simulation(Dueling_DDQN_Agent)

### 4. Watch Trained Agent

In [ ]:
def watch_trained(Agent, env):
    # Load in trained Agent
    agent = Agent(state_size, action_size, 88)
    agent.qnetwork_local.load_state_dict(torch.load(f"C:\Dev\Python\RL\\Navigation_Project\p1_navigation\checkpoints\\{agent}.pth"))

    # Run the simulation and print the score
    score = 0
    eps = 0 #No exploration on trained agent
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations[0]
    while True:
        action = int(agent.act(state,eps))
        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        score += reward
        state = next_state
        if done:
            break

    print(f"Episode Score: {score}")

watch_trained(DQN_Agent, env)
env.close()


### 5. Show the results

In [ ]:
def graph_results(Agents, save_graph=True):
    """ Used to graph the results of agents from simulations

    :param agent_scores: list of Class Agents you want to graph results from
    :param save_graph: bool used to save the graph in images
    """
    # Load scores for given Agents
    agent_scores = []
    for agent in Agents:
        if agent == DQN_Agent:
            class_name = "DQN_Agent"
        elif agent == DoubleDQN_Agent:
            class_name = "Double_DQN_Agent"
        elif agent == Dueling_DDQN_Agent:
            class_name = "Dueling_DDQN_Agent"
        with open(f"C:\Dev\Python\RL\\Navigation_Project\p1_navigation\saved_scores\\{class_name}.txt", 'r') as f:
            scores = f.read().splitlines()
            scores = [float(score) for score in scores]
            agent_scores.append((class_name, scores))

    # Setup up graph
    fig, ax = plt.subplots()
    ax.set(xlabel="Episode #", ylabel='Score', title="Navigation Agent Learning Rate")
    ax.grid()
    save_name = ""
    # Graph all the results
    for name, score in agent_scores:
        #Plot Scores
        ax.plot(np.arange(len(score)), score, label=f"{name}")
        save_name += f"{name}_"
        # Plot rolling average
        rolling_average = np.convolve(score, np.ones(100)/100)
        rolling_average = rolling_average[:-100] #Removes tail that is window_size < 100
        ax.plot(np.arange(len(rolling_average)), rolling_average, label=f"Average per 100 episodes")
    # Plot solved line
    ax.plot(np.arange(len(agent_scores[0][1])), np.ones(len(agent_scores[0][1]))*13, color='black', linestyle='dashed', label="Solved")
    ax.legend()

    if save_graph:
        filepath = f"C:\Dev\Python\RL\\Navigation_Project\p1_navigation\images\\{save_name}.png"
        fig.savefig(filepath)
    plt.show()

# Setup what sort of graphs you want
graph_dqn = False
graph_double_dqn = False
graph_dueling = False
graph_comparison = False
if graph_dqn:
    graph_results([DQN_Agent])
if graph_double_dqn:
    graph_results([DoubleDQN_Agent])
if graph_dueling:
    graph_results([Dueling_DDQN_Agent])
if graph_comparison:
    graph_results([DQN_Agent, DoubleDQN_Agent, Dueling_DDQN_Agent])